# MLE

Le but du MLE est de trouver une première aproximation des points. Cependant cette aproximation tend à trop coller aux données du modèle.

On importe les bibliothèques :

In [2]:
!py -m pip install numpy
import numpy as np


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Comme montrer dans l'article, pour déterminer la distance, on n'est pas forcé de prendre la distance euclidienne. On peut prendre comme distance le plus court chemin entre chaque famille i et j.

Pour cela on définie les matrices $(Y_r)_{r = 1,...,n}$ ($n$ : nombre de famille) tel que : 

$Y_1 = Y$ ($Y$ notre matrice d'adjacence initiale) $\\$
$Y_{r+1} = Y_r.Y$

On a alors :

$Y_r[i,j]>0$ ssi il existe un chemin de taille r entre i et j. En effet : 

$$Y_r[i,j]>0 \iff \overset{n}{\underset{k=1}{\sum}} Y_r[i,k].Y[k,j]>0$$
$$Y_r[i,j]>0 \iff \exist k \in \{1,...,n\} Y_r[i,k].Y[k,j]>0$$ 
(On a même $Y_r[i,j]$ qui nous donne le nombre de chemin de taille $r$ entre i et j, mais nous ne sommes intéressé que par l'existance d'un chemin).

On peut ensuite en déduire notre matrice des distances : 
$$D[i,j]=min \left\{ min \left\{ r, Y_r[i,j]>0\right\}, n \right\}$$

In [ ]:
def distance(Y):
    """
    Calcule la distance entre les noeuds comme le nombre minimal
    d'arêtes entre les noeuds
    (renvoie n=nombre de sommets si pas de lien)
    """
    n = np.shape(Y)[0]
    # On détermine notre Y_{1}
    Yr = np.copy(Y)
    # On modifie progressivement notre distance : on a déjà les noeuds à distance de 1 les uns des autres
    D = np.copy(Y)
    D = Y * (Y == 1) + n * (Y == 0)
    for r in range(2, n):
        # On détermine notre Y_{r+1}
        Yr = np.dot(Yr, Y)
        # On modifie notre distance pour les paires de noeuds n'ayant pas de distance plus courte
        D += (r - n) * ((Yr > 0) & (D == n))
    # On met bien des 0 sur la diagonale de D
    np.fill_diagonal(D, 0)
    return D

On peut ensuite déterminer une fonction nous donnant un nuage de point dans $\mathbb{R}^k$ tel que les points collent au plus aux distance grâce à la fonction `svd` du module `numpy.linalg` :

In [ ]:
def MLE(Y, k=2):
    D = distance(Y)
    Z = np.linalg.svd(D)[0][:, :k] # à changer
    # On vérifie que les points sont biens centrés ?
    return Z